In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [1]:
!pip install pyspark==2.3.2
import pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from pyspark.sql import SparkSession

In [6]:
#from pyspark import SparkSession
#from pyspark import SparkContext, SparkConf

In [7]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf



# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-07-30 15:06:24--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  72.6MB/s    in 0.2s    

2022-07-30 15:06:25 (72.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://dc4e-35-245-0-139.ngrok.io


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bostan").getOrCreate()

In [10]:
! ls

boston.csv  ngrok  ngrok-stable-linux-amd64.zip  sample_data


In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

data = spark.read.csv("/content/boston.csv",inferSchema=True, header =True)

In [13]:
data.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)



##**1. Read the given CSV file in a Hive table & 2.Read the data from Hive table as spark dataframe** 

In [15]:
data.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

In [16]:
data.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT',
 'MV']

In [23]:
data.head(5)

[Row(CRIM=0.00632, ZN=18.0, INDUS=2.309999943, CHAS=0, NOX=0.537999988, RM=6.574999809, AGE=65.19999695, DIS=4.090000153, RAD=1, TAX=296, PT=15.30000019, B=396.8999939, LSTAT=4.980000019, MV=24.0),
 Row(CRIM=0.027310001, ZN=0.0, INDUS=7.070000172, CHAS=0, NOX=0.469000012, RM=6.421000004, AGE=78.90000153, DIS=4.967100143, RAD=2, TAX=242, PT=17.79999924, B=396.8999939, LSTAT=9.140000343, MV=21.60000038),
 Row(CRIM=0.02729, ZN=0.0, INDUS=7.070000172, CHAS=0, NOX=0.469000012, RM=7.184999943, AGE=61.09999847, DIS=4.967100143, RAD=2, TAX=242, PT=17.79999924, B=392.8299866, LSTAT=4.03000021, MV=34.70000076),
 Row(CRIM=0.032370001, ZN=0.0, INDUS=2.180000067, CHAS=0, NOX=0.458000004, RM=6.998000145, AGE=45.79999924, DIS=6.062200069, RAD=3, TAX=222, PT=18.70000076, B=394.6300049, LSTAT=2.940000057, MV=33.40000153),
 Row(CRIM=0.069049999, ZN=0.0, INDUS=2.180000067, CHAS=0, NOX=0.458000004, RM=7.146999836, AGE=54.20000076, DIS=6.062200069, RAD=3, TAX=222, PT=18.70000076, B=396.8999939, LSTAT=5.329

##**3. Get the correlation between dependent and independent**

In [26]:
from pyspark.sql.functions import *
for i in data.columns:
  print("Correlation of the independent variables",i, data.stat.corr('MV', i))


Correlation of the independent variables CRIM -0.3883046116575088
Correlation of the independent variables ZN 0.36044534463752903
Correlation of the independent variables INDUS -0.48372517128143383
Correlation of the independent variables CHAS 0.17526017775291847
Correlation of the independent variables NOX -0.4273207763683772
Correlation of the independent variables RM 0.695359937127267
Correlation of the independent variables AGE -0.37695456714288667
Correlation of the independent variables DIS 0.24992873873512172
Correlation of the independent variables RAD -0.3816262315669168
Correlation of the independent variables TAX -0.46853593528654536
Correlation of the independent variables PT -0.5077867038116085
Correlation of the independent variables B 0.3334608226834164
Correlation of the independent variables LSTAT -0.7376627294671615
Correlation of the independent variables MV 1.0


##**4. Build a linear regression model to predict house price**

In [27]:
data.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT',
 'MV']

In [28]:
from pyspark.ml.feature import VectorAssembler
vassembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')


In [29]:
vhouse_df = vassembler.transform(data)

In [30]:
vhouse_df =vhouse_df.select(['features', 'MV'])
vhouse_df.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



In [31]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [32]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.005747254626117237,0.0,0.3501390510685801,-7.178995538315047,4.0289477045914825,0.0,-0.7467296093306518,0.0,-0.0009943388305484761,-0.8031996943116276,0.006777179042201903,-0.5237880610959548]
Intercept: 23.253799957498103


#**5. Evaluate the Linear Regression model by getting the RMSE and R-squared values**

In [33]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.669603
r2: 0.725113


In [34]:
train_df.describe().show()

+-------+------------------+
|summary|                MV|
+-------+------------------+
|  count|               336|
|   mean|22.665476188901785|
| stddev| 8.919691374502252|
|    min|       6.300000191|
|    max|              50.0|
+-------+------------------+



In [35]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|31.267680328805803|32.70000076|[0.01301,35.0,1.5...|
| 29.15162430950057|35.40000153|[0.01311,90.0,1.2...|
|30.333815655757615|31.60000038|[0.01432,100.0,1....|
|26.671244787858047|       24.5|[0.01501,80.0,2.0...|
|38.674207163601714|       50.0|[0.01501,90.0,1.2...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.676646


In [37]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 11
objectiveHistory: [0.5000000000000004, 0.4328471234260123, 0.23835785776184892, 0.21503818961934104, 0.18071974727122747, 0.17792646309692864, 0.17701071734859017, 0.1758602410241501, 0.1749631424532184, 0.17414004392867796, 0.17408892683440108]
+-------------------+
|          residuals|
+-------------------+
|  -6.42927519593178|
| 2.2089330302578034|
| -5.096547670226336|
|  2.377995973721781|
| 12.509334832661828|
|-1.0480254119478367|
|  9.781227702278791|
|  4.812619778111724|
|  2.742331080544872|
|-2.0435772256050235|
|0.15252986203527286|
|  6.675520930211711|
|0.38978727033869376|
| -9.626294549511815|
| -3.317614790154188|
| 3.5604123067545785|
| -3.812498379014361|
| 2.1806537605836986|
|-1.9165961064605987|
| 2.6533641911809696|
+-------------------+
only showing top 20 rows

